<a href="https://colab.research.google.com/github/basvoju/few_shot/blob/project_data/project_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.2 MB/s eta 0:00:00


In [3]:
import numpy as np 
from transformers import BertTokenizerFast 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 

In [4]:
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
os.chdir("/content/drive/MyDrive/data")
os.listdir()

['text2tech_train.json', 'text2tech_test.json']

In [7]:
import pandas as pd
train_data = pd.read_json('/content/drive/MyDrive/data/text2tech_train.json', lines=True)
test_data = pd.read_json('/content/drive/MyDrive/data/text2tech_test.json', lines=True)
train_data

,tokens,ner_tags,id
0,"[Quantum, communication, holds, the, promise, ...","[4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"[The, invention, relates, to, a, lost, foam, c...","[0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...",1
2,"[Organic, solar, cells, have, the, potential, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",2
3,"[Manufacturing, electronic, devices, by, print...","[0, 4, 4, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, ...",3
4,"[Graphic, representation, of, optical, signals...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 2, ...",4


In [8]:
test_data

,tokens,ner_tags,id
0,"[Large, -, scale, simulations, of, thermal, we...","[1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, ...",5
1,"[A, component, of, a, computer, -, aided, manu...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...",6
2,"[Organic, solar, cells, have, the, potential, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",7
3,"[Since, programming, is, an, extremely, time, ...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8
4,"[As, much, mystique, as, they, 've, been, give...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...",9


In [9]:
train_data['tokens']

0    [Quantum, communication, holds, the, promise, ...
1    [The, invention, relates, to, a, lost, foam, c...
2    [Organic, solar, cells, have, the, potential, ...
3    [Manufacturing, electronic, devices, by, print...
4    [Graphic, representation, of, optical, signals...
Name: tokens, dtype: object

In [10]:
train_data["tokens"][0]

['Quantum',
 'communication',
 'holds',
 'the',
 'promise',
 'of',
 'creating',
 'disruptive',
 'technologies',
 'that',
 'will',
 'play',
 'an',
 'essential',
 'role',
 'in',
 'future',
 'communication',
 'networks',
 '.',
 'For',
 'example',
 ',',
 'the',
 'study',
 'of',
 'quantum',
 'communication',
 'complexity',
 'has',
 'shown',
 'that',
 'quantum',
 'communication',
 'allows',
 'exponential',
 'reductions',
 'in',
 'the',
 'information',
 'that',
 'must',
 'be',
 'transmitted',
 'to',
 'solve',
 'distributed',
 'computational',
 'tasks',
 '.',
 'Recently',
 ',',
 'protocols',
 'that',
 'realize',
 'this',
 'advantage',
 'using',
 'optical',
 'implementations',
 'have',
 'been',
 'proposed',
 '.',
 'Here',
 'we',
 'report',
 'a',
 'proof',
 'of',
 'concept',
 'experimental',
 'demonstration',
 'of',
 'a',
 'quantum',
 'fingerprinting',
 'system',
 'that',
 'is',
 'capable',
 'of',
 'transmitting',
 'less',
 'information',
 'than',
 'the',
 'best',
 'known',
 'classical',
 'proto

In [11]:
train_data["ner_tags"]

0    [4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
3    [0, 4, 4, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 2, ...
Name: ner_tags, dtype: object

In [12]:
test_data['tokens']

0    [Large, -, scale, simulations, of, thermal, we...
1    [A, component, of, a, computer, -, aided, manu...
2    [Organic, solar, cells, have, the, potential, ...
3    [Since, programming, is, an, extremely, time, ...
4    [As, much, mystique, as, they, 've, been, give...
Name: tokens, dtype: object

In [13]:
test_data['ner_tags']

0    [1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...
2    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
3    [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...
Name: ner_tags, dtype: object

In [14]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
tokenized_input = tokenizer(train_data["tokens"][0], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

word_ids = tokenized_input.word_ids()

print(word_ids)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 76, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 128, 129, 130, 131, 132, 133, 134, 135, 136, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, None]


In [16]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'quantum',
 'communication',
 'holds',
 'the',
 'promise',
 'of',
 'creating',
 'disrupt',
 '##ive',
 'technologies',
 'that',
 'will',
 'play',
 'an',
 'essential',
 'role',
 'in',
 'future',
 'communication',
 'networks',
 '.',
 'for',
 'example',
 ',',
 'the',
 'study',
 'of',
 'quantum',
 'communication',
 'complexity',
 'has',
 'shown',
 'that',
 'quantum',
 'communication',
 'allows',
 'exponential',
 'reductions',
 'in',
 'the',
 'information',
 'that',
 'must',
 'be',
 'transmitted',
 'to',
 'solve',
 'distributed',
 'computational',
 'tasks',
 '.',
 'recently',
 ',',
 'protocols',
 'that',
 'realize',
 'this',
 'advantage',
 'using',
 'optical',
 'implementations',
 'have',
 'been',
 'proposed',
 '.',
 'here',
 'we',
 'report',
 'a',
 'proof',
 'of',
 'concept',
 'experimental',
 'demonstration',
 'of',
 'a',
 'quantum',
 'finger',
 '##print',
 '##ing',
 'system',
 'that',
 'is',
 'capable',
 'of',
 'transmitting',
 'less',
 'information',
 'than',
 'the',
 'best',


In [17]:
len(train_data['ner_tags']), len(tokenized_input["input_ids"])


(5, 160)

In [18]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [19]:
train_data['tokens'][4:5]

4    [Graphic, representation, of, optical, signals...
Name: tokens, dtype: object

In [20]:
q = tokenize_and_align_labels(train_data[4:5]) 
print(q)

ValueError: ignored